# 6.3 LinGAMを用いた因果探索

本ファイルは、6.3節の内容となります。LiNGAMを実装しながらその内容の理解を深めていきます。

## プログラム実行前の設定など

In [0]:
# 乱数のシードを固定
import random
import numpy as np

random.seed(1234)
np.random.seed(1234)


In [0]:
# 使用するパッケージ（ライブラリと関数）を定義
import pandas as pd

# データ生成

## モデル
x1 = 3×x2 + ex1

x2 = ex2

x3 = 2×x1 + 4×x2 + ex3
 


In [3]:
# データ数
num_data = 200

# 非ガウスのノイズ
ex1 = 2*(np.random.rand(num_data)-0.5)  # -1.0から1.0
ex2 = 2*(np.random.rand(num_data)-0.5)
ex3 = 2*(np.random.rand(num_data)-0.5)

# データ生成
x2 = ex2
x1 = 3*x2 + ex1
x3 = 2*x1 + 4*x2 + ex3

# 表にまとめる
df = pd.DataFrame({"x1": x1, "x2": x2, "x3": x3})
df.head()


,x1,x2,x3
0,2.257272,0.958078,8.776842
1,2.531611,0.762464,8.561263
2,0.641547,0.255364,1.341902
3,3.153636,0.860973,9.322791
4,1.908691,0.449580,5.776675


## 独立成分分析を実施

In [4]:
# 独立成分分析はscikit-learnの関数を使用します
from sklearn.decomposition import FastICA
# https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.FastICA.html

ica = FastICA(random_state=1234).fit(df)

# ICAで求めた行列A
A_ica = ica.mixing_

# 行列Aの逆行列を求める
A_ica_inv = np.linalg.pinv(A_ica)
print(A_ica_inv)


[[-0.23203107 -0.4635971   0.1154553 ]
 [-0.02158245  0.12961253  0.00557934]
 [-0.11326384  0.40437635 -0.00563091]]


## 行列A_invを求め、行の順番と大きさを調整

プログラムの参考

https://qiita.com/m__k/items/bd87c063a7496897ba7c

In [5]:
!pip install munkres
from munkres import Munkres
from copy import deepcopy


In [6]:
# 実装の参考
# [5] Qiita：LiNGAMモデルの推定方法について
# https://qiita.com/m__k/items/bd87c063a7496897ba7c

# ①「行の順番を変換」→対角成分の絶対値を最大にする
# （元のA^-1の対角成分は必ず0ではないので）

# 絶対値の逆数にして対角成分の和を最小にする問題に置き換える
A_ica_inv_small = 1 / np.abs(A_ica_inv)

# 対角成分の和を最小にする行の入れ替え順を求める
m = Munkres()  # ハンガリアン法
ixs = np.vstack(m.compute(deepcopy(A_ica_inv_small)))

# 求めた順番で変換
ixs = ixs[np.argsort(ixs[:, 0]), :]
ixs_perm = ixs[:, 1]
A_ica_inv_perm = np.zeros_like(A_ica_inv)
A_ica_inv_perm[ixs_perm] = A_ica_inv
print(A_ica_inv_perm)


[[-0.11326384  0.40437635 -0.00563091]
 [-0.02158245  0.12961253  0.00557934]
 [-0.23203107 -0.4635971   0.1154553 ]]


In [7]:
# 並び替わった順番
print(ixs)

[[0 2]
 [1 1]
 [2 0]]


In [8]:
# ②「行の大きさを調整」
D = np.diag(A_ica_inv_perm)[:, np.newaxis]  # D倍されているDを求める
A_ica_inv_perm_D = A_ica_inv_perm / D
print(A_ica_inv_perm_D)


[[ 1.         -3.57021564  0.04971498]
 [-0.16651518  1.          0.0430463 ]
 [-2.00970483 -4.01538182  1.        ]]


In [9]:
# ③「B=I-A_inv」
B_est = np.eye(3) - A_ica_inv_perm_D
print(B_est)


[[ 0.          3.57021564 -0.04971498]
 [ 0.16651518  0.         -0.0430463 ]
 [ 2.00970483  4.01538182  0.        ]]


In [10]:
# ①上側成分の0になるはずの数（3×3であれば3個、4×4であれば6個と、対角成分の上側の要素数分）、絶対値が小さい成分を0にする
# ②変数の順番を入れ替えて、下三角行列になるかを確かめる、
# 実装の参考
# [5] Qiita：LiNGAMモデルの推定方法について
# https://qiita.com/m__k/items/bd87c063a7496897ba7c

def _slttestperm(b_i):
# b_iの行を並び替えて下三角行列にできるかどうかチェック
    n = b_i.shape[0]
    remnodes = np.arange(n)
    b_rem = deepcopy(b_i)
    p = list() 

    for i in range(n):
        # 成分が全て0である行番号のリスト
        ixs = np.where(np.sum(np.abs(b_rem), axis=1) < 1e-12)[0]

        if len(ixs) == 0:
            return None
        else:
            ix = ixs[0]
            p.append(remnodes[ix])

            #　成分が全て0である行を削除
            remnodes = np.hstack((remnodes[:ix], remnodes[(ix + 1):]))
            ixs = np.hstack((np.arange(ix), np.arange(ix + 1, len(b_rem))))
            b_rem = b_rem[ixs, :]
            b_rem = b_rem[:, ixs]

    return np.array(p)

b = B_est
n = b.shape[0]
assert(b.shape == (n, n))

ixs = np.argsort(np.abs(b).ravel())

for i in range(int(n * (n + 1) / 2) - 1, (n * n) - 1):
    b_i = deepcopy(b)
    b_i.ravel()[ixs[:i]] = 0
    ixs_perm = _slttestperm(b_i)
    if ixs_perm is not None:
        b_opt = deepcopy(b)
        b_opt = b_opt[ixs_perm, :]
        b_opt = b_opt[:, ixs_perm]
        break
b_csl = np.tril(b_opt, -1)
b_csl[ixs_perm, :] = deepcopy(b_csl)
b_csl[:, ixs_perm] = deepcopy(b_csl)

B_est1 = b_csl
print(B_est1)



[[0.         3.57021564 0.        ]
 [0.         0.         0.        ]
 [2.00970483 4.01538182 0.        ]]


## Bの非ゼロ要素を求め直す

In [11]:
# scikit-learnから線形回帰をimport
from sklearn.linear_model import LinearRegression

# 説明変数
X1 = df[["x2"]]
X3 = df[["x1", "x2"]]

# 被説明変数（目的変数）
# df["x1"]
# df["x3"]

# 回帰の実施
reg1 = LinearRegression().fit(X1, df["x1"])
reg3 = LinearRegression().fit(X3, df["x3"])

# 回帰した結果の係数を出力
print("係数：", reg1.coef_)
print("係数：", reg3.coef_)


係数： [3.14642595]
係数： [1.96164568 4.11256441]


以上